In [1]:
# import packages
import h5py
import numpy as np
import torch
from mirtorch.linear import Sense, Diff3dgram
from mirtorch.alg.cg import CG
import os
import sys

# select device
if torch.cuda.is_available():
    print('CUDA is available')
    device0 = torch.device('cuda:2')
else:
    print('CUDA is not available')
    device0 = torch.device('cpu')

CUDA is available


In [ ]:
# load in the GRE data
fname = '/home/djfrey/data/lps_fmri_20250331/gre.h5'

with h5py.File(fname, 'r') as h5_file:
    kdata = h5_file['kdata/real'][:] + 1j * h5_file['kdata/imag'][:] # kspace data
    msk = h5_file['msk'][:] # sampling mask
    fov = h5_file['seq_args/fov'][0][0] # fov (cm)
    N = int(h5_file['seq_args/N'][0][0]) # 3D matrix size
    Nacs = int(h5_file['seq_args/Nacs'][0][0]) # 3D matrix size
    ncoil = int(h5_file['ncoil'][0][0]) # number of coils

# reshape into tensors
kdata = torch.tensor(kdata).reshape(1,ncoil,N,N,N) # C x Kz x Ky x Kx

In [ ]:
# import BART toolbox
bart_path = os.getenv('BART_TOOLBOX_PATH')
if bart_path:
    sys.path.append(os.path.join(bart_path, 'python'))
else:
    raise EnvironmentError("BART_TOOLBOX_PATH environment variable is not set")
from bart import bart

# calculate the coil sensitivity maps
smaps = bart(1, f'ecalib -b0 -m1 -r{Nacs}', kdata.squeeze(0).permute(3,2,1,0).numpy()) # X x Y x Z x C
smaps = torch.tensor(smaps).permute(3,2,1,0) # C x Z x Y x X

# save to h5 file in the same directory as fname
output_path = os.path.join(os.path.dirname(fname), 'smaps.h5')
with h5py.File(output_path, 'w') as smaps_h5:
    smaps_h5.create_dataset('real', data=smaps.real.numpy())
    smaps_h5.create_dataset('imag', data=smaps.imag.numpy())